# Mesh generation for tutorial 03

In [ ]:
import numpy as np
import pygmsh
from mpi4py import MPI
from dolfinx import cpp
from dolfinx.mesh import create_mesh, create_meshtags
from dolfinx.io import ufl_mesh_from_gmsh, XDMFFile
from dolfinx.plotting import plot

In [ ]:
assert MPI.COMM_WORLD.size == 1, "This mesh generation notebook is supposed to be run in serial"

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create pygmsh mesh

In [ ]:
geom = pygmsh.geo.Geometry()
p0 = geom.add_point([0.0, 0.0, 0.0], lcar)
p1 = geom.add_point([0.0, +r, 0.0], lcar)
p2 = geom.add_point([0.0, -r, 0.0], lcar)
c0 = geom.add_circle_arc(p1, p0, p2)
c1 = geom.add_circle_arc(p2, p0, p1)
l0 = geom.add_line(p2, p1)
geom.add_physical([c0, c1], label=1)
geom.add_physical(l0, label=2)
line_loop_left = geom.add_line_loop([c0, l0])
line_loop_right = geom.add_line_loop([c1, -l0])
semicircle_left = geom.add_plane_surface(line_loop_left)
semicircle_right = geom.add_plane_surface(line_loop_right)
geom.add_physical(semicircle_left, label=11)
geom.add_physical(semicircle_right, label=12)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Convert to a dolfinx mesh

In [ ]:
cells, x = pygmsh_mesh.cells_dict["triangle"], pygmsh_mesh.points[:, :2]
mesh = create_mesh(MPI.COMM_WORLD, cells, x, ufl_mesh_from_gmsh("triangle", x.shape[1]))

In [ ]:
plot(mesh)

### Extract subdomains as MeshTags

In [ ]:
subdomains_entities = cells
subdomains_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["triangle"] - 10
subdomains = create_meshtags(mesh, mesh.topology.dim,
                             cpp.graph.AdjacencyList_int32(subdomains_entities), np.int32(subdomains_values))
subdomains.name = "subdomains"

### Extract boundaries as MeshTags

In [ ]:
boundaries_entities = pygmsh_mesh.cells_dict["line"]
boundaries_values = pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]
mesh.topology.create_connectivity(mesh.topology.dim - 1, 0)
boundaries = create_meshtags(mesh, mesh.topology.dim - 1,
                             cpp.graph.AdjacencyList_int32(boundaries_entities), np.int32(boundaries_values))
boundaries.name = "boundaries"

### Save mesh, subdomains and boundaries

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "circle.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(subdomains)
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    file.write_meshtags(boundaries)